In [ ]:
import csv
import pandas as pd
from googleapiclient.discovery import build

# Replace with your own API key
API_KEY = 'api key vikash'
YOUTUBE_API_SERVICE_NAME = 'youtube'
YOUTUBE_API_VERSION = 'v3'

def get_youtube_service():
    return build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, developerKey=API_KEY)

def get_video_details():
    youtube = get_youtube_service()
    
    # Fetch the first 5 videos from the playlist or channel
    request = youtube.search().list(
        part='snippet',
        type='video',
        order='date',
        maxResults=5
    )
    response = request.execute()
    
    videos = []
    
    for item in response['items']:
        video_id = item['id']['videoId']
        title = item['snippet']['title']
        thumbnail_url = item['snippet']['thumbnails']['high']['url']
        video_url = f'https://www.youtube.com/watch?v={video_id}'
        
        # Get video statistics
        stats_request = youtube.videos().list(
            part='statistics, contentDetails',
            id=video_id
        )
        stats_response = stats_request.execute()
        views = stats_response['items'][0]['statistics'].get('viewCount', 'N/A')
        upload_time = stats_response['items'][0]['snippet'].get('publishedAt', 'N/A')

        videos.append({
            'Title': title,
            'Video URL': video_url,
            'Thumbnail URL': thumbnail_url,
            'Views': views,
            'Upload Time': upload_time
        })
    
    return videos

def save_to_csv(videos):
    df = pd.DataFrame(videos)
    df.to_csv('youtube_videos.csv', index=False)

if __name__ == '__main__':
    videos = get_video_details()
    save_to_csv(videos)
    print("Data saved to youtube_videos.csv")
